In [ ]:

# libraries
import os
import time
import subprocess
import numpy as np
import pandas as pd
import ast
import cv2
import PIL.Image
import matplotlib.pyplot as plt
import timm
% matplotlib inline
import seaborn as sns
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
# from warmup_scheduler import GradualWarmupScheduler
import albumentations
import torch.cuda.amp as amp
# import segmentation_models_pytorch as smp
from tqdm import tqdm

scaler = amp.GradScaler()
# device = torch.device('cuda')
from torchvision.io import read_image

TRAIN_DIR = 'dataset/train/train/'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
# create pandas dataframe to store the results
df = None
df = pd.DataFrame(columns=['filepath', 'id', 'labels'])

DOGS_TRAIN = os.path.join(TRAIN_DIR, 'dogs')
CATS_TRAIN = os.path.join(TRAIN_DIR, 'cats')


In [ ]:

for image in os.listdir(DOGS_TRAIN):
    # print(os.path.join(DOGS_TRAIN, image))
    file_path = os.path.join(DOGS_TRAIN, image)
    df = df.append({
        'filepath': file_path,
        'id': image, 'labels': 1
    }, ignore_index=True)

for image in os.listdir(CATS_TRAIN):
    # print(os.path.join(CATS_TRAIN, image))
    file_path = os.path.join(CATS_TRAIN, image)
    df = df.append({
        'filepath': file_path,
        'id': image, 'labels': 0
    }, ignore_index=True)
# save the dataframe to csv
df.to_csv('train_labels.csv', index=False)

In [ ]:

df['labels'].value_counts()

In [ ]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')
# plot the distribution of labels
sns.countplot(train_df['labels'])
# as pie
plt.show()
plt.pie(train_df['labels'].value_counts(), labels=train_df['labels'].value_counts().index, autopct='%1.1f%%')
plt.show()
plt.pie(val_df['labels'].value_counts(), labels=val_df['labels'].value_counts().index, autopct='%1.1f%%', shadow=True)
plt.show()


In [ ]:
# plot the first batch of images
fig, ax = plt.subplots(1, len(train_features), figsize=(20, 4))
for i in range(len(train_features)):
    ax[i].imshow(train_features[i].permute(1, 2, 0).numpy())
    ax[i].set_title(train_labels[i].item())
    ax[i].axis('off')
